In [1]:
import os
import subprocess
import pandas as pd
from rdkit import Chem

conf_list = ["output_4FOB_backbone_72.txt","output_4FOB_backbone_108.txt","output_4FOB_backbone_2321.txt","4FOB.txt"]

def extract_number(filename):
    # Extract digits from the filename
    digits = ''.join(filter(str.isdigit, filename))
    # Convert extracted digits to an integer
    return int(digits) if digits else 0

def extract_affinity_from_sdf(sdf_file):
    # Load SDF file and extract affinity value #LATER TO BE USED IN EXTRACTING AFFINITY FROM DOCKING RESULT
    mol_supplier = Chem.SDMolSupplier(sdf_file)
    if mol_supplier is not None and len(mol_supplier) > 0:
        mol = mol_supplier[0]
        try:
            affinity_field = mol.GetProp('minimizedAffinity')  # Assuming affinity is stored as a property
            affinity_value = float(affinity_field)
            return affinity_value
        except KeyError:
            print(f"Warning: 'minimizedAffinity' property not found in {sdf_file}")
            return None
        except AttributeError:
            print(f"Warning: Molecule in {sdf_file} is None or invalid")
            return None
    else:
        print(f"Warning: No valid molecules found in {sdf_file}")
        return None

In [2]:
from config import processed_dir
ligands_dir = f"{processed_dir}/ligands/selected"
# Get list of ligand filenames
files = [filename for filename in os.listdir(ligands_dir) if filename.endswith(".pdbqt")]

# Initialize a DataFrame to store affinity values
ligands = sorted(files, key=extract_number)  # Extract ligand names without extension

affinity_df = pd.DataFrame(index=ligands, columns=conf_list)

csv_file_path = f"{processed_dir}/Docking_results.csv"



In [3]:
ligands_dir

'/home/bogrum/project_files/4FOB/processed/ligands/selected'

In [12]:
for a in conf_list:
    for b in sorted(files, key=extract_number):
        # Print the filename
        ligdirectory = os.path.join(ligands_dir, b)
        outputsdf = f"conf_{a[21:-4]}_ligno_{b[7:-6]}_out_for_padding_1.sdf"
        outputsdfdirectory = os.path.join(ligands_dir, outputsdf)
        outputlog = a.replace(".txt", "_dock_log.log")
        outputlogdirectory = os.path.join(ligands_dir, outputlog)
        print(ligdirectory, outputsdf, outputsdfdirectory, outputlog, outputlogdirectory)  
        
        # Execute subprocess to run smina command
        subprocess.call(f"smina --config {a} --ligand {ligdirectory} --out {outputsdfdirectory} --cpu 12 --log {outputlogdirectory}", shell=True)
        
        # Extract affinity value from output sdf file
        try:
            affinity_value = extract_affinity_from_sdf(outputsdfdirectory)
            if affinity_value is not None:
                ligname = b.replace(".txt", "")
                affinity_df.loc[ligname, a] = affinity_value
                affinity_df.to_csv(csv_file_path, index_label="Ligand", sep="\t", float_format='%.5f')
        except Exception as e:
            print(f"Warning: An error occurred while processing {outputsdfdirectory}: {str(e)}")
            continue

/home/bogrum/project_files/4FOB/processed/ligands/selected/output_1.pdbqt conf_72_ligno_1_out_for_padding_1.sdf /home/bogrum/project_files/4FOB/processed/ligands/selected/conf_72_ligno_1_out_for_padding_1.sdf output_4FOB_backbone_72_dock_log.log /home/bogrum/project_files/4FOB/processed/ligands/selected/output_4FOB_backbone_72_dock_log.log
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8

*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
*** Open Babel Error  in TetStereoToWedgeHash
  Failed to set stereochemistry as unable to find an available bond
[13:05:32] Explicit valence for atom # 7 N, 4, is greater than permitted
[13:05:32] ERROR: Could not sanitize molecule ending on line 112
[13:05:32] ERROR: Explicit valence for atom # 7 N, 4, is greater

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -2048536844

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------

[13:06:52] Explicit valence for atom # 4 N, 4, is greater than permitted
[13:06:52] ERROR: Could not sanitize molecule ending on line 73
[13:06:52] ERROR: Explicit valence for atom # 4 N, 4, is greater than permitted


   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1980165980

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


...

[13:39:25] WARNING: not removing hydrogen atom without neighbors


   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -1933447425

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------

[13:40:12] Explicit valence for atom # 17 N, 4, is greater than permitted
[13:40:12] ERROR: Could not sanitize molecule ending on line 87
[13:40:12] ERROR: Explicit valence for atom # 17 N, 4, is greater than permitted


   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 2076447184

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------




Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: Search space dimensions should be positive


Usage error: